In [40]:
import pandas as pd
import os
path_to_results = os.environ['HOME']+'/projects/ra_recommendator_conrec/results/'

# All experiments
# experiment = "20190717_1108_5p_100" - images: done
# experiment = "20190714_1211_6p" - images: done
# experiment = "20190714_1213_5p" - images: done
# experiment = "20190714_1231_7p" - images: done
# experiment = "20190714_1231_8p" - images: done
# experiment = "20190714_1232_9p" - images: done
# experiment = "20190714_1233_10p" - images: done
# experiment = "20190714_1234_3p"
# experiment = "20190714_1235_4p"
# experiment = "20190714_1236_2p"


nb_of_participants = experiment[14:-1]
scenarios = ['A', 'B', 'C', 'D']
data = {}
for s in scenarios:
    data[s] = pd.read_csv(path_to_results+experiment+'/recall_for_scenario_'+s+'.csv', sep='\t', index_col=0)
data['A'].head()

,top 5,top 10,top 15,top 20,top 25,top 30,top 35,top 40,top 45,top 50,...,top 105,top 110,top 115,top 120,top 125,top 130,top 135,top 140,top 145,top 150
q_id,,,,,,,,,,,,,,,,,,,,,
12962,0.2,0.2,0.2,0.3,0.3,0.4,0.5,0.6,0.6,0.6,...,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8
90536,0.1,0.2,0.2,0.2,0.2,0.2,0.2,0.3,0.4,0.4,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
195459,0.3,0.4,0.7,0.8,0.8,0.8,0.8,0.8,0.8,0.8,...,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8
9797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.1,...,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3,0.3
212614,0.2,0.3,0.3,0.3,0.3,0.3,0.3,0.4,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [41]:
import plotly.graph_objects as go

up_to_top100 = data[scenario].columns[:-10]

def figure_maker(scenario, columns):
    fig = go.Figure()
    for col in data[scenario][columns]:
        fig.add_trace(go.Box(
            y=data[scenario][col],
            name=col, boxpoints='outliers', 
            marker_color='rgb(9,56,125)', line_color='rgb(9,56,125)'
        ))

    fig.update_layout(
        title_text="Recall Scenario "+scenario+" - "+ nb_of_participants +" participants, all questions",
        showlegend=False,
        height=500)
    
    return fig
    

#for scenario in scenarios:
for scenario in scenarios:
    figure_maker(scenario,up_to_top100).show()


In [87]:
# all questions in which any of the scenarios performed poorly, ie. Recall@60 <= 0.2
questions_with_bad_performance = [9135, 9197, 9251, 9273, 9277, 9427, 9606, 9615, 9620, 9653, 9684, 9750, 9796, 10006, 10013, 10020, 10027, 10072, 10105, 10120, 10149, 10238, 10280, 10284, 10313, 10323, 10342]

bad_performance = []

for scenario in scenarios:
    bad_performance.append(data[scenario][data[scenario].index.isin(questions_with_bad_performance)]['top 60'])
df = pd.DataFrame(bad_performance)
df = df.transpose()
df.columns = scenarios

def give_arrow(diff):
    # Big difference (> 0.4)
    if abs(diff) > 0.4:
        # Positive difference
        if diff > 0:
            # df.at[index,'Q+'] = '⬆⬆'
            return '⬆⬆'
        # Negative difference
        else:
            # df.at[index,'Q+'] = '⬇⬇'
            return '⬇⬇'
    # Small difference (<= 0.4)        
    else:
        # Positive difference
        if diff > 0:
            # df.at[index,'Q+'] = '⬆'
            return '⬆'
        # Negative difference        
        elif diff < 0:
            # df.at[index,'Q+'] = '⬇' 
            return '⬇' 
        # No difference
        else:
            # df.at[index,'Q+'] = '-' 
            return '-'

for index, row in df.iterrows():
    max_val = max([row['B'],row['C'],row['D']])
    diff = max_val - row['A']
    df.at[index,'Q+'] = give_arrow(diff)
    df.at[index,'U+'] = give_arrow(diff)    

df.sort_index()


,A,B,C,D,Q+,U+
q_id,,,,,,
9135,0.2,0.2,1.0,0.8,⬆⬆,⬆⬆
9197,0.8,0.6,0.6,0.4,⬇,⬇
9251,0.8,0.6,0.8,0.4,-,-
9273,0.2,0.2,0.6,0.4,⬆,⬆
9277,0.2,0.0,0.4,0.4,⬆,⬆
9427,0.4,0.2,1.0,0.6,⬆⬆,⬆⬆
9606,0.6,0.4,0.8,0.6,⬆,⬆
9615,0.2,0.0,0.4,0.4,⬆,⬆
9620,0.8,0.8,0.8,0.8,-,-


In [67]:
"""
TODO:
- Subplots para los boxplot
- Exportar para que se vean en el README.md
- Tabla de las flechas
"""

'\nTODO:\n- Subplots para los boxplot\n- Exportar para que se vean en el README.md\n- Tabla de las flechas\n'